In [ ]:
import json
import random
import uuid
import pandas as pd
from datetime import datetime, timedelta
from kafka import KafkaProducer
import pytz

producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

sent_events = []

# 한국 시간으로 현재 시간 설정
korea_tz = pytz.timezone('Asia/Seoul')
current_time = datetime.now(korea_tz)

users = {
    "111": {"view": 5, "cart": 2, "purchase": 1},
    "222": {"view": 2, "cart": 1, "purchase": 0},  
    "333": {"view": 1, "cart": 0, "purchase": 0}
}

print("Hourly feature 테스트용 이벤트 생성 시작")
print(f"한국 시간 기준: {current_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")

for user_id, pattern in users.items():
    print(f"사용자 {user_id} 이벤트 생성:")
    
    for event_type, count in pattern.items():
        for i in range(count):
            event = {
                "event_time": current_time.isoformat(),  # 모든 이벤트 동일한 시간
                "event_type": event_type,
                "product_id": str(random.randint(10000000, 99999999)),
                "category_id": str(random.randint(1000000000000000000, 9999999999999999999)),
                "category_code": random.choice(["electronics", "beauty", None]),
                "brand": random.choice(["apple", "samsung", "nike"]),
                "price": round(random.uniform(10, 500), 2),
                "user_id": user_id,
                "user_session": str(uuid.uuid4())
            }
            
            producer.send("ecommerce-events", value=event)
            sent_events.append(event)
            print(f"  {event_type} - {current_time.strftime('%H:%M:%S')}")

producer.flush() # 모든 메시지가 전송될 때까지 대기
sent_df = pd.DataFrame(sent_events)

Hourly feature 테스트용 이벤트 생성 시작
한국 시간 기준: 2025-07-03 21:37:48 KST
사용자 111 이벤트 생성:
  view - 21:37:48
  view - 21:37:58
  view - 21:38:08
  view - 21:38:18
  view - 21:38:28
  cart - 21:38:38
  cart - 21:38:48
  purchase - 21:38:58
사용자 222 이벤트 생성:
  view - 21:39:08
  view - 21:39:18
  cart - 21:39:28
사용자 333 이벤트 생성:
  view - 21:39:38


In [2]:
# Redis 연결 및 키 확인
import redis
import json
from pprint import pprint

# Redis 클라이언트 생성
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

try:
    r.ping()
    print('Redis 연결 성공')
except Exception as e:
    print(f"Redis 연결 실패: {e}")


Redis 연결 성공


In [3]:
# 1. 모든 user_features 키 조회
user_keys = r.keys("*")
for key in user_keys[:10]:  # 처음 10개만 표시
    print(f"- {key}")


In [9]:

# State 보존 테스트: 추가 이벤트 전송
from datetime import datetime, timedelta
import pytz
import json
import uuid

korea_tz = pytz.timezone('Asia/Seoul')
future_time = datetime.now(korea_tz)

test_event = {
    "event_time": future_time.isoformat(),
    "event_type": "view",
    "product_id": "99999999",
    "category_id": "9999999999999999999",
    "category_code": "electronics",
    "brand": "apple", 
    "price": 100.0,
    "user_id": "111",  # 기존 사용자
    "user_session": str(uuid.uuid4())
}

producer.send("ecommerce-events", value=test_event)
producer.flush()

In [16]:
# 사용자 111의 현재 시간 1시간 전부터 이벤트 조회
from kafka import KafkaConsumer
import json
from datetime import datetime, timedelta
from collections import defaultdict

user_id = "111"
current_time = datetime.now()
# start_time = current_time - timedelta(hours=1)  # 현재 시간 1시간 전
start_time = datetime(2025, 7, 3, 21, 37, 0, 0)
duration_seconds = 3

print(f"=== {start_time.strftime('%Y-%m-%d %H:%M:%S')} 이후 사용자 {user_id} 이벤트 ===")
print(f"현재 시간: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"최대 {duration_seconds}초간 읽습니다...")

# Kafka Consumer 설정
consumer = KafkaConsumer(
    'ecommerce-events',  # 올바른 토픽명
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8')),
    consumer_timeout_ms=duration_seconds * 1000,  # 최대 읽기 시간
    auto_offset_reset='earliest'  # 처음부터 읽기
)

events = []
event_type_counts = defaultdict(int)

try:
    for message in consumer:
        event = message.value
        
        # Kafka 메시지 발행 시간 확인
        kafka_time = datetime.fromtimestamp(message.timestamp / 1000.0)
        
        # 카프카 발행 시간으로 필터링 (1시간 전부터)
        if kafka_time < start_time:
            continue
        
        # 사용자 ID 필터링 (올바른 필드명)
        if event.get('user_id') != user_id:
            continue
            
        # 이벤트 시간 파싱
        try:
            if event.get('event_time'):
                # ISO 포맷 시간 파싱
                event_time_str = event.get('event_time')
                if '+' in event_time_str:
                    event_time = datetime.fromisoformat(event_time_str.replace('+09:00', ''))
                else:
                    event_time = datetime.fromisoformat(event_time_str)
            else:
                event_time = kafka_time
        except:
            event_time = kafka_time
        
        # 이벤트 정보 저장
        events.append({
            'kafka_time': kafka_time,
            'event_time': event_time,
            'event_type': event.get('event_type'),  # 올바른 필드명
            'user_session': event.get('user_session'),  # 올바른 필드명
        })
        event_type_counts[event.get('event_type')] += 1
        
        # 이벤트 출력
        print(f"Kafka: {kafka_time.strftime('%H:%M:%S.%f')[:-3]} | "
              f"Event: {event_time.strftime('%H:%M:%S.%f')[:-3]} | "
              f"{event.get('event_type')} | "
              f"Product: {event.get('product_id')}")
              
except Exception as e:
    print(f"Consumer 종료: {e}")
finally:
    consumer.close()

# 요약 출력
print(f"\n=== 요약 ===")
print(f"총 {len(events)}개 이벤트 발견")

if event_type_counts:
    print(f"\n이벤트 타입별 집계:")
    for event_type, count in sorted(event_type_counts.items()):
        print(f"  {event_type}: {count}개")


=== 2025-07-03 21:37:00 이후 사용자 111 이벤트 ===
현재 시간: 2025-07-03 21:43:49
최대 3초간 읽습니다...
Kafka: 21:37:48.430 | Event: 21:37:48.429 | view | Product: 11997188
Kafka: 21:37:48.430 | Event: 21:37:58.429 | view | Product: 18744325
Kafka: 21:37:48.430 | Event: 21:38:08.429 | view | Product: 46672814
Kafka: 21:37:48.430 | Event: 21:38:18.429 | view | Product: 74994405
Kafka: 21:37:48.430 | Event: 21:38:28.429 | view | Product: 79591538
Kafka: 21:37:48.431 | Event: 21:38:38.429 | cart | Product: 67247315
Kafka: 21:37:48.431 | Event: 21:38:48.429 | cart | Product: 75348890
Kafka: 21:37:48.431 | Event: 21:38:58.429 | purchase | Product: 86300472
Kafka: 21:37:59.673 | Event: 21:37:59.672 | view | Product: 99999999

=== 요약 ===
총 9개 이벤트 발견

이벤트 타입별 집계:
  cart: 2개
  purchase: 1개
  view: 6개


In [4]:
r.hgetall("hourly_features:111")

{}